In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

In [2]:
circ = 26700 # m
c    = 3e8 # m / s
mu_tau = 2.2e-6 # s

n_steps = 1000

e_max = 7000 # GeV
e_inj = 450 
e_per_turn = 14.0
sc_to_pls_ratio = 16 / 3.8
magnet_fraction = 0.85

def makePlot(x_vals, y_vals, xlab, ylab):
    plt.plot(x_vals, y_vals)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.ylim(bottom=0)
    plt.grid()
    plt.show()

def getEVsTime(e_inj, e_per_turn, e_max, circ):
    
    # Calculate in terms of time
    t_per_turn = circ / c
    e_per_sec = e_per_turn / t_per_turn
    t_max = (e_max - e_inj)/e_per_sec
    
    # Get values to plot
    t_vals = np.linspace(0, t_max, n_steps)
    e_vals = t_vals*e_per_sec + e_inj
    
    return t_vals, e_vals

def getFracRemainingVsTime(t_vals, e_vals):
    
    # Calculate the gamma for each timestep
    t_max = t_vals[-1]
    t_steps = []
    for i, val in enumerate(t_vals):
        if i > 0:
            t_steps.append(t_vals[i]-t_vals[i-1])
        else:
            t_steps.append(0)
    g_vals = e_vals / .106
    
    # Calculate the fraction that decays in each time step
    t_step_proper_vals = t_steps / g_vals
    decay_frac_vals = np.exp(-t_step_proper_vals/mu_tau)
    
    # Multiply those fractions to get the cumulative decay
    remaining_vals = []
    for i, val in enumerate(decay_frac_vals):
        if i == 0:
            remaining_vals.append(1)
        else:
            remaining_vals.append(val*remaining_vals[i-1])
    return remaining_vals
    
def getBVsTime(e_vals, circ):
    
    r = circ / (2*np.pi) # in m
    q = 1.6e-19 # in Coulombs
    
    p_vals = np.sqrt((e_vals)**2 - (0.106)**2) * 1.6e-10 / c # in kg m / s
    p_vals
    B_vals = p_vals / (magnet_fraction*r*q) # 0.85 assumes 15% of your length doesn't have magnets
    #return 2*np.pi*e_vals/(0.3*circ) # From paper, returns the same
    
    return B_vals

def plotSingle(e_inj, e_per_turn, circ, e_max, sc_to_pls_ratio):
    
    # Create plot of energy v. time
    t_vals, e_vals = getEVsTime(e_inj, e_per_turn, e_max, circ)
    makePlot(t_vals, e_vals, "Time [s]", "Energy [GeV]")
    
    # Create plot of decay fraction v. time and E
    remaining_vals = getFracRemainingVsTime(t_vals, e_vals)
    makePlot(t_vals, remaining_vals, "Time [s]", "Fraction of beam remaining")
    makePlot(e_vals, remaining_vals, "Energy [GeV]", "Fraction of beam remaining")
    
    # Create a plot of B-field v. time
    B_vals = getBVsTime(e_vals, circ)
    makePlot(t_vals, B_vals, "Time [s]", "Average B-field [T]")
    
    # See https://iopscience.iop.org/article/10.1088/1748-0221/13/10/T10003/pdf for pls ratio stuff
    ramp = (B_vals[-1]-B_vals[0])/(t_vals[-1]-t_vals[0])
    E_ratio = e_max / e_inj
    pls_ratio = sc_to_pls_ratio*(E_ratio-1)/(E_ratio+1) # Get fraction of L_pls / L_sc
    pls_to_active_ratio = 1 / (1+1/pls_ratio)  # Fraction of active magnet length that are pulsed
    pls_frac = pls_to_active_ratio*magnet_fraction # Fraction of full circumference with pulsed magnets
    print("Final raction remaining: %.3f"%remaining_vals[-1])
    print("Total ramp time: %.3f ms"%(t_vals[-1]*1000))
    print("Max average B field: %.3f T"%B_vals[-1])
    print("Average ramp speed: %.3f T/s"%ramp)
    print("Length of pulsed magnet: %.3f m"%(circ*pls_frac))
    print("Length of fixed magnet: %.3f m"%(circ*(magnet_fraction-pls_frac)))
    print("Ramp speed of pulsed only: %.3f T/s"%(ramp/pls_to_active_ratio)) 
    
def plotDouble(e_inj0, e_per_turn0, circ0, e_inj1, e_per_turn1, circ1, e_max):
    
    # Create the list of the first times and energies
    t_vals0, e_vals0 = getEVsTime(e_inj0, e_per_turn0, e_inj1, circ0)
    t_max0 = t_vals0[-1]
    
    # Create the list of the second times and energies
    t_vals1, e_vals1 = getEVsTime(e_inj1, e_per_turn1, e_max, circ1)
    t_vals1 += t_max0

    # Put them together and plot
    t_vals = np.concatenate((t_vals0, t_vals1), axis=0)
    e_vals = np.concatenate((e_vals0, e_vals1), axis=0)
    makePlot(t_vals, e_vals, "Time [s]", "Energy [GeV]")
    
    # Get decay fraction from this
    remaining_vals = getFracRemainingVsTime(t_vals, e_vals)
    makePlot(t_vals, remaining_vals, "Time [s]", "Fraction of beam remaining")
    makePlot(e_vals, remaining_vals, "Energy [GeV]", "Fraction of beam remaining")
    
    # Make B-field plot
    B_vals = getBVsTime(e_vals, circ)
    makePlot(t_vals, B_vals, "Time [s]", "Average B-field [T]")
    
    print("Fraction remaining: ", remaining_vals[-1])
    print("Total first ramp time: %.3f ms"%(t_vals0[-1]*1000))
    print("Total second ramp time: %.3f ms"%(t_vals1[-1]*1000))

In [3]:
interact(plotSingle, e_inj=e_inj, e_per_turn=e_per_turn, circ=circ, e_max=e_max, sc_to_pls_ratio=sc_to_pls_ratio);

interactive(children=(IntSlider(value=450, description='e_inj', max=1350, min=-450), FloatSlider(value=14.0, d…

In [4]:
interact(plotSingle, e_inj=30, e_per_turn=3.7, circ=6900, e_max=450, sc_to_pls_ratio=8/0.8);

interactive(children=(IntSlider(value=30, description='e_inj', max=90, min=-30), FloatSlider(value=3.7, descri…

In [5]:
interact(plotDouble, e_inj0=30, 
         e_per_turn0=3.7,
         circ0 = 6900,
         e_inj1=450,
         e_per_turn1=14,
         circ1 = 26700,
         e_max = 7000,
        );

interactive(children=(IntSlider(value=30, description='e_inj0', max=90, min=-30), FloatSlider(value=3.7, descr…